### 0. Import Libs

In [1]:
# kernel must be msd2
import sys;sys.path.append('../')
import os
import pandas as pd
import constant as c
import csv
import re
from util.epsg_transform import *
from shapely.geometry import *

### 1. Get Data

In [2]:
data_nm = 'popse'
dataset_nm = 'popse.csv'
data_path = os.path.join(c.SSD_PATH, data_nm, dataset_nm)
popse = pd.read_csv(data_path, quoting=csv.QUOTE_NONE, encoding = 'cp949')
popse.columns = c.POPSE_GEN_COLS
popse = popse.drop(columns = "NN")
popse = popse.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
popse = popse[popse['단위'] == '명']
popse = popse[popse['구분별'] == '합계']
popse = popse.drop(columns=['구분별', '항목','단위'])
sggbjds = []
gu_nm = ''
for i in range(len(popse)):
    word = popse.iloc[i]['동별']
    if word[-1] == '구':
        gu_nm = word
        sggbjds.append(gu_nm)
    else:
        sggbjds.append(f'{gu_nm} {word}')
popse['구동명'] = sggbjds
int_cols = ['구동명', '202001', '202101', '202201']
popse = popse[int_cols]

### 2. Get Coordinates

In [3]:
admdongxy_path = os.path.join(c.SSD_PATH, 'admdong_coord.xlsx')
admdongxy = pd.read_excel(admdongxy_path, sheet_name = '서울특별시') #, encoding = 'utf-8')
def remove_je(string):
    return re.sub(r'제(\d+)', r'\1', string)
admdongxy['읍면동/구'] = admdongxy['읍면동/구'].astype(str)
admdongxy['읍면동/구'] = admdongxy['읍면동/구'].apply(remove_je)
admdongxy = admdongxy[admdongxy['읍면동/구'] != 'nan']
admdongxy['구동명'] = admdongxy['시군구'] + ' ' + admdongxy['읍면동/구']
df_popse = pd.merge(popse, admdongxy, on = '구동명', how = 'left')
df_popse = df_popse[['시군구', '읍면동/구', '구동명', '202001', '202101', '202201', '위도', '경도']]
df_popse = df_popse.rename(columns={'위도': 'y', '경도': 'x', '시군구': 'sgg_nm', '읍면동/구': 'bjd_nm', '구동명': 'sggbjd_nm'})
df_popse = df_popse.dropna(subset=['bjd_nm'])
df_popse['x'] = pd.to_numeric(df_popse['x'], errors='coerce')
df_popse['y'] = pd.to_numeric(df_popse['y'], errors='coerce')
x_5179, y_5179 = coord_tranformation(list(df_popse.x), list(df_popse.y), "epsg:4326", "epsg:5179")
df_popse['x_5179'] = x_5179
df_popse['y_5179'] = y_5179
df_popse.to_csv('../asset/preprocess/features/df_popse.csv')

In [4]:
# Read df_popse
df_popse = pd.read_csv('../asset/preprocess/features/df_popse.csv', index_col = 0)